In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/home/user01/llama2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = 'llama2-v3'
device_map = {"": 0}
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
        load_in_4bit=use_4bit,
        bnb_4bit_quant_type=bnb_4bit_quant_type,
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=use_nested_quant,
)
    # Load base model
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left" # Fix weird overflow issue with fp16 training


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.27s/it]


In [3]:
import pandas as pd


test_data = pd.read_csv('3. data_sentiment_without_label.csv')
negative = [0 for i in range(len(test_data))]
neutral = [0 for i in range(len(test_data))]
positive = [0 for i in range(len(test_data))]


In [4]:

logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
for i, d in enumerate(test_data['tweet']):
    prompt = f"""what is the sentiment of the following sentence.
    Your respose should be only one word: positive, negative or neutral.
    Sentence: {d}

    """
    encode = tokenizer.encode(prompt, add_special_tokens=True)
    length = len(encode)
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=length+12)

    result = pipe(f"<s>[INST] {prompt} [/INST]")
    # print(result[0]['generated_text'])
    match result[0]['generated_text'].split()[-1]:
        case 'منفی':
            negative[i] = 1
        case 'خنثی':
            neutral[i] = 1
        case 'مثبت':
            positive[i] = 1

/home/user01/llama2/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [7]:
test_data['negative'] = negative
test_data['positive'] = positive
test_data['neutral'] = neutral

In [10]:
new = ['local_id', 'tweet', 'negative', 'neutral', 'positive']
test_data = test_data[new]
test_data.head(100)

,local_id,tweet,negative,neutral,positive
0,983807604982996995,دوستان بارسایی. \nحالا بشینید و فوتبال خوب تما...,0,0,1
1,1058990279196323840,اگر بازی استقلال. جوبیلو ایواتا توی استادیوم آ...,0,0,1
2,1202268201859588097,یکی از کلان پروژه‌های دوران امامت #امام_حسن_عس...,0,0,1
3,1206882143718268928,جریان #غربگرا در حالی برای پوشش ناکارآمدی‌اش، ...,0,0,1
4,1226924630809726980,زمانی که به‌نام #وحدت ایرادات اساسی یک ساختار ...,0,0,1
...,...,...,...,...,...
95,1434536742573412352,عمیقا معتقدم بهترین گزینه موجود استانداری #خوز...,0,0,1
96,1435180059929759744,امیدوارم بچه حزب‌الهی‌ها اونقدر در آمریکاستیزی...,0,0,1
97,1435719814585196547,وقتی با شخص #مناسب هستی\nمجبور نیستی برای خوشح...,0,0,1
98,1438216598939213833,حرفهایی امشب #مددی در مورد #استقلال دقیقا منو...,0,0,1


In [11]:
test_data.to_csv('test_output.csv', index=False)

In [11]:
prompt = """
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item
The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.
Review text: '''من کلا از شیائومی راضی هستم؛ قبلا ترازو و دستگاه تصفیه هواشو گرفتم، الان مسواکشو خریداری کردم. پیشنهاد میکنم'''"""
run(model, prompt)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.44s/it]
/home/user01/llama2/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] 
Identify the following items from the review text:
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item
The review is delimited with triple backticks. Format your response as a JSON object with "Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" as the value.
Make your response as short as possible.
Format the Anger value as a boolean.
Review text: '''من کلا از شیائومی راضی هستم؛ قبلا ترازو و دستگاه تصفیه هواشو گرفتم، الان مسواکشو خریداری کردم. پیشنهاد میکنم''' [/INST] جان را در نظر بگیرید، او یک مسواکشو خریداری کرده است. او رایی دارد و قبلا ترازو و دستگاه تصفیه هواشو گرفت. او پیشنهاد می کند. او رایی دارد. او رایی دارد. او رایی دارد. او رایی دارد. او رایی دارد. او رایی دارد. او رایی دارد. او رایی دارد. او رایی دا


In [1]:
from run_model import run

model = 'llama2-v2'

prompt = 'مدل زبانی چیست؟'
run(model, prompt)

/home/user01/llama2/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.99s/it]
/home/user01/llama2/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


<s>[INST] مدل زبانی چیست؟ [/INST]
